### Mesh processing for SimuCell3D

This notebook outlines how to generate a geometry for the SimuCell3D.

In [1]:
import os
import sys
import numpy as np
from MeshPrep import isolate_filtered_meshes
from MeshPrep import mesh_process_clean, string_to_array

In [2]:
sys.path.append('/nas/groups/iber/Users/Federico_Carrara/Statistics_Collection/EpiStats/src/statistics_collection/')
from StatsAnalytics import prepare_df

#### Cleaning the labels

The labels generated from manual curation should be processesed before passing through the geometry generation. Strongly recommended step. Generates meshes which may be used for manual cell patch selection using paraview.

In [25]:
root = '../outputs/outputs_v3/output_lung_new_sample_b_curated_segmentation_central_crop_relabel_seq_s_10_e_6_d_8/'

In [21]:
voxel_resolution = np.array([0.1625, 0.1625, 0.25])
label_path = os.path.join(root, 'processed_labels.tif')

In [ ]:
# from VoxelProcessing import full_label_processing

# output_folder='path/folder/to/save/processed/labels'
# cell_info = full_label_processing(labeled_img=label_path, voxel_resolution=voxel_resolution, output_folder=output_folder, smoothing_iterations=10)
# label_path = os.path.join(output_folder, "processed_labels.npy")
# filtered_cell_list = cell_info[2]

#### Select labels for Simulations

To obtain labels for simulation one may use napari and select cells manually, or use meshes, and paraview. View how to use paraview [here](Tutorials/LabelSelection.md).

You may want to isolate the cells that do not touch the border instead. To isolate these use the following script and use Paraview's extract functionality as outlined.

In [27]:
from FilterMeshes import get_valid_cell_ids

filtered_cell_list = get_valid_cell_ids(os.path.join(root, 'cell_stats/stats_dataset_lung_bronchiole.csv'))

In [22]:
mesh_path = os.path.join(root, 'cell_meshes')
# This is where the isolated meshes will be saved.
isolated_mesh_path = '../../Meshes_for_Simulation/examples/cell_clump_esophagus' 

isolate_filtered_meshes(mesh_path, isolated_mesh_path, filtered_cell_list)

Converting files to .vtk: 100%|██████████| 52/52 [00:18<00:00,  2.89it/s]


Now you can open the `.vtk` mesh files in paraview and select a clump of cells for simulation.

#### Mesh Cleaning for SimuCell3D

Produce meshes for SimuCell3D simulation framework. Ensure that the path to labels is of cleaned, processed labels. 

In [30]:
root = '../outputs/outputs_v3/output_esophagus_Z2_curated_crop_s_10_e_6_d_8/'
voxel_resolution = np.array([0.1625, 0.1625, 0.25])
label_path = os.path.join(root, 'processed_labels.tif')

label_list = string_to_array("100 101 104 124 128 129 54 62 64 66 74 80 85 87 89 96 ")
output_dir = '../../Meshes_for_Simulation/examples/cell_clump_esophagus/cell_clumps/clean_clump_16_cells/clean_meshes'

# Call the mesh_process_clean function
mesh_process_clean(
    label_path=label_path, 
    output_dir=output_dir, 
    label_list=label_list, 
    voxel_resolution=voxel_resolution, 
    scale_factor=1e-6, 
    min_edge_length=0.5,
    make_shell=True
)

-------------------------------------------
Creating meshes from labeled img...


Converting labels to meshes: 100%|██████████| 16/16 [01:48<00:00,  6.79s/it]


-------------------------------------------
First mesh cleaning...


Cleaning non-mainfold meshes:   0%|          | 0/18 [00:00<?, ?it/s]

Removed 16 non-manifold faces
INFO- Loaded 26043 vertices and 52070 faces.


...


0% done 
INFO- ********* ITERATION 0 *********
INFO- Removing degeneracies...
INFO- Removing self-intersections...

94 % done   
INFO- No intersections detected.

0% done 
-------------------------------------------
Getting `.vtk` files...


Converting files to .vtk: 100%|██████████| 17/17 [00:02<00:00,  6.52it/s]


-------------------------------------------
Preparing file for simulation...


Merging .vtk files: 100%|██████████| 17/17 [00:00<00:00, 22.51it/s]


'../../Meshes_for_Simulation/examples/cell_clump_esophagus/cell_clumps/clean_clump_16_cells/clean_meshes/vtk_files/merged.vtk'